# Integrate own Splitter
As mentioned in the previous section, to make this as easy to use as possible, this notebook provide a template for integrating own Splitter in the pipeline.

In [ ]:
from flask import Flask, request, Response, jsonify
import json
import spacy

app = Flask(__name__)

print("Loading Custom Splitter")


You can modify the process method, to make it produce documents as needed.
in the default case, this method, copies the document as many times as there is child nodes.
another use case would be to split the document acording to the meaning of a mention, and pass all "People" to the first document, and "objects" to the second document and so on..

**NOTE**: Please make sure that you generate as many documents as there are child nodes, otherwise the assertion in the index method will be triggered.

In [ ]:
# Example copy split
def process(document, number_of_child_nodes):

    return [ document for i in range(0, number_of_child_nodes) ]


In [ ]:
@app.route('/', methods=['get', 'post'])
def index():
    print("Incoming request:")
    req = json.loads(request.data)
    document = req['document']

    number_of_child_nodes = 0
    for child in req['pipelineConfig']['connections']:
        if child['source'] == req['componentId']:
            number_of_child_nodes+=1
    if req['componentId'] in req['pipelineConfig']['endComponents']:
        number_of_child_nodes += 1


    documents = process(document, number_of_child_nodes)

    assert len(documents) == number_of_child_nodes

    documents_dict_array = [{'document': doc} for doc in documents]


    return jsonify(
            {'documents' : documents_dict_array,
            'pipelineConfig' : req['pipelineConfig'],
            'componentId' : req['componentId']}
            )

In [ ]:
class LoggingMiddleware(object):
    def __init__(self, app):
        self._app = app

    def __call__(self, env, resp):
        errorlog = env['wsgi.errors']
        #pprint.pprint(('REQUEST', env), stream=errorlog)

        def log_response(status, headers, *args):
            #pprint.pprint(('RESPONSE', status, headers), stream=errorlog)
            return resp(status, headers, *args)

        return self._app(env, log_response)


# Run at flask startup (https://stackoverflow.com/a/55573732)
with app.app_context():
    pass

You can change the port of the web service here, after running the following code the webservice will be available.

**NOTE**: It only accepts get and post requests, trying to open the web page will result in an error with code 500.

In [ ]:
if __name__ == '__main__':
    # MAKE SURE YOU DON'T USE THE SAME PORT ON THE OTHER PIPELINE COMPONENTS
    port = 5004 
    print("Running app... on port: ", port)
    app.wsgi_app = LoggingMiddleware(app.wsgi_app)
    app.run(host='0.0.0.0', port=port)